# 🚛 Example 3: Transportation Problem using Gurobi

This example demonstrates solving a **classical transportation problem** using **Gurobi** in Python. The goal is to minimize the total cost of transporting goods from supply sources to demand destinations.

Transportation problems are fundamental in **operations research** and are widely used in logistics, supply chain management, and distribution planning.

---

## 🔢 Mathematical Formulation

We want to:

**Minimize:**
$$\sum_{i \in I} \sum_{j \in J} c_{ij} \cdot x_{ij}$$

**Subject to:**

**Supply Constraints:**
$$\sum_{j \in J} x_{ij} \leq s_i, \quad \forall i \in I$$

**Demand Constraints:**
$$\sum_{i \in I} x_{ij} = d_j, \quad \forall j \in J$$

**Non-negativity:**
$$x_{ij} \geq 0, \quad \forall i \in I, \forall j \in J$$

**Where:**
- $x_{ij}$ = units transported from source $i$ to destination $j$
- $c_{ij}$ = cost per unit from source $i$ to destination $j$
- $s_i$ = supply capacity at source $i$
- $d_j$ = demand requirement at destination $j$
- $I$ = set of sources, $J$ = set of destinations

---

## 📊 Problem Instance

**Sources:** S1, S2  
**Destinations:** D1, D2, D3

**Supply Capacities:**
- S1: 20 units
- S2: 30 units
- **Total Supply:** 50 units

**Demand Requirements:**
- D1: 10 units  
- D2: 25 units
- D3: 15 units
- **Total Demand:** 50 units

**Transportation Costs (per unit):**

| From/To | D1 | D2 | D3 |
|---------|----|----|----|
| **S1**  | 8  | 6  | 10 |
| **S2**  | 9  | 12 | 7  |

---

🎯 This is a **balanced transportation problem** since total supply equals total demand (50 = 50).

The model will find the optimal allocation that minimizes total transportation cost while satisfying all supply and demand constraints.

In [ ]:
# Transportation model

import gurobipy as gp
from gurobipy import GRB

# Define the data
sources = ['S1', 'S2']
demands = ['D1', 'D2', 'D3']
supply = {'S1': 20, 'S2': 30}
demand = {'D1': 10, 'D2': 25, 'D3': 15}
cost = {
    ('S1', 'D1'): 8, ('S1', 'D2'): 6, ('S1', 'D3'): 10,
    ('S2', 'D1'): 9, ('S2', 'D2'): 12, ('S2', 'D3'): 7
}

# Create a new model
model = gp.Model("transportation")

# Create variables
x = model.addVars(sources, demands, name="x")

# Set objective
model.setObjective(gp.quicksum(cost[s, d] * x[s, d] for s in sources for d in demands), GRB.MINIMIZE)

# Add supply constraints
model.addConstrs((gp.quicksum(x[s, d] for d in demands) <= supply[s] for s in sources), name="Supply")

# Add demand constraints
model.addConstrs((gp.quicksum(x[s, d] for s in sources) == demand[d] for d in demands), name="Demand")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    for s in sources:
        for d in demands:
            if x[s, d].x > 0:
                print(f"Transport {x[s, d].x} units from {s} to {d} at cost {cost[s, d]} per unit")
else:
    print("No optimal solution found")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xab1bdb4e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 5 rows and 6 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.750000000e+02
Transport 20.0 units from S1 to D2 at cost 6 per unit
Transport 10.0 units from S2 to D1 at cost 9 per unit
Transport 5.0 units from S2 to D2 at cost 12 per unit
Transport 15.0 units from S2 to D3 at